## Original version of SIREN on PyTorch

#### Author: Denys Herasymuk

In [39]:
from IPython.display import Image, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [40]:
!pip install siren-pytorch
clear_output()

In [41]:
!pip install pysdf
clear_output()

In [42]:
!pip install trimesh
clear_output()

In [43]:
!pip install Rtree
clear_output()

In [58]:
import os
import time
import pysdf
import trimesh
from pysdf import SDF
from tqdm import tqdm

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from siren_pytorch import SirenNet

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import numpy as np
import skimage
import matplotlib.pyplot as plt

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
DATA_DIR = '/content/drive/MyDrive/UCU/4course_1term/Computer_Vision/Practice/data/'

## Helper functions for the model training

In [61]:
def get_near_surface_points_and_truth(mesh, num_points = 2048):
  f = SDF(mesh.vertices, mesh.faces)
  
  points = f.sample_surface(num_points)
  points += np.random.normal(scale=0.01, size=(num_points, 3))
  ground_truth = f(points)
  
  return points, ground_truth


def get_near_surface_points_and_truth_cuda(mesh, num_points = 2048):
  points, ground_truth = get_near_surface_points_and_truth(mesh)
  points = torch.cuda.FloatTensor(points)
  ground_truth = torch.cuda.FloatTensor(ground_truth)

  return points.cuda(), ground_truth.cuda()


def get_random_points_and_truth(mesh, num_points = 2048):
  points = np.random.uniform(-1.0,1.0,(num_points,3))
  ground_truth = trimesh.proximity.signed_distance(mesh, points)
  
  return points, ground_truth


def get_random_points_and_truth_cuda(mesh, num_points = 2048):
  points, ground_truth = get_random_points_and_truth(mesh)
  points = torch.cuda.FloatTensor(points)
  ground_truth = torch.cuda.FloatTensor(ground_truth)

  return points.cuda(), ground_truth.cuda()


def train_model(obj_siren, mesh):
  obj_siren = obj_siren.cuda()

  total_steps = 3000 # Since the whole object is our dataset, this just means the number of gradient descent steps.
  steps_til_summary = 50
  loss1, loss2 = -1.0, -1.0
  execution_time_lst = []
  # In this training cycle, I use two losses to optimize both occupancy F1 scores.
  # Note that I change points for training to avoid overfitting. 
  # Actually, it helps significantly to decrease loss over time.
  for step in tqdm(range(total_steps)):
      start_time = time.time()
      points_near_surface, ground_truth_near_surface = get_near_surface_points_and_truth_cuda(mesh)

      optim = torch.optim.Adam(lr=1e-5, params=obj_siren.parameters())
      sdf_near_surface = obj_siren(points_near_surface)
      loss1 = ((sdf_near_surface - ground_truth_near_surface)**2).mean()
      optim.zero_grad()
      loss1.backward()
      optim.step()
        
      if not step % steps_til_summary:
          print(f'\n\nStep {step}, loss_near_surface: {loss1}, loss_random: {loss2}\n\n')
          points_random, ground_truth_random = get_random_points_and_truth_cuda(mesh)

      sdf_random = obj_siren(points_random)
      optim = torch.optim.Adam(lr=3e-6, params=obj_siren.parameters())
      loss2 = ((sdf_random - ground_truth_random)**2).mean()
      optim.zero_grad()
      loss2.backward()
      optim.step()
      end_time = time.time()
      execution_time_lst.append(end_time - start_time)

  print(f'\n\nStep {total_steps}, loss_near_surface: {loss1}, loss_random: {loss2}\n\n')

  return obj_siren, execution_time_lst


In [62]:
def run_model(obj_idx):
  # Train the model
  obj_siren = SirenNet(
      dim_in = 3,
      dim_hidden = 240,
      dim_out = 1,
      num_layers = 5,
      final_activation = nn.Identity(),
      w0_initial = 30
  )
  mesh = trimesh.load_mesh(DATA_DIR + f'test_task_meshes/{obj_idx}.obj')
  obj_siren, execution_time_lst = train_model(obj_siren, mesh)

  model_save_path = DATA_DIR + f'siren_model/obj_{obj_idx}.pt'
  torch.save(obj_siren.state_dict(), model_save_path)

  # Calculate metrics for the model
  memory_in_kb = os.path.getsize(model_save_path) / 1024
  avg_time_secs = sum(execution_time_lst) / len(execution_time_lst)

  print(f'Avg time in seconds: {avg_time_secs}')
  print(f'Memory usage in Kb: {memory_in_kb}')

  return {
    'avg_time_secs': avg_time_secs,
    'memory_usage': memory_in_kb
  }


## Train models

In [63]:
models_metrics = {}

In [64]:
obj_idx = 1
models_metrics[f'obj_{obj_idx}'] = run_model(obj_idx)

  0%|          | 0/3000 [00:00<?, ?it/s]



Step 0, loss_near_surface: 0.7218019962310791, loss_random: -1.0




  2%|▏         | 48/3000 [00:05<01:44, 28.37it/s]



Step 50, loss_near_surface: 0.21346572041511536, loss_random: 0.9188302159309387




  3%|▎         | 99/3000 [00:09<01:30, 31.96it/s]

...



Step 2900, loss_near_surface: 0.003771301591768861, loss_random: 0.15448066592216492




 98%|█████████▊| 2947/3000 [03:46<00:01, 29.58it/s]



Step 2950, loss_near_surface: 0.0036455783993005753, loss_random: 0.14840179681777954




100%|██████████| 3000/3000 [03:50<00:00, 13.03it/s]



Step 3000, loss_near_surface: 0.003837680211290717, loss_random: 0.15215127170085907


Avg time in seconds: 0.0762248059908549
Memory usage in Kb: 912.0009765625


In [65]:
obj_idx = 2
models_metrics[f'obj_{obj_idx}'] = run_model(obj_idx)

  0%|          | 0/3000 [00:00<?, ?it/s]



Step 0, loss_near_surface: 0.5600359439849854, loss_random: -1.0




  2%|▏         | 49/3000 [00:12<04:17, 11.48it/s]



Step 50, loss_near_surface: 0.19194167852401733, loss_random: 0.5526593327522278




  3%|▎         | 99/3000 [00:24<04:20, 11.13it/s]

...



Step 2900, loss_near_surface: 0.0038738136645406485, loss_random: 0.16781310737133026




 98%|█████████▊| 2949/3000 [11:57<00:04, 11.50it/s]



Step 2950, loss_near_surface: 0.004474093671888113, loss_random: 0.16608433425426483




100%|██████████| 3000/3000 [12:10<00:00,  4.11it/s]



Step 3000, loss_near_surface: 0.004452974535524845, loss_random: 0.16153205931186676


Avg time in seconds: 0.24256762337684631
Memory usage in Kb: 912.0009765625


In [66]:
obj_idx = 3
models_metrics[f'obj_{obj_idx}'] = run_model(obj_idx)

  0%|          | 0/3000 [00:00<?, ?it/s]



Step 0, loss_near_surface: 0.6701544523239136, loss_random: -1.0




  2%|▏         | 50/3000 [00:14<06:14,  7.88it/s]



Step 50, loss_near_surface: 0.22744402289390564, loss_random: 0.5561544895172119




  3%|▎         | 100/3000 [00:29<06:03,  7.98it/s]

...



Step 2900, loss_near_surface: 0.00794959906488657, loss_random: 0.147847518324852




 98%|█████████▊| 2950/3000 [13:31<00:06,  7.81it/s]



Step 2950, loss_near_surface: 0.008034972473978996, loss_random: 0.13921386003494263




100%|██████████| 3000/3000 [13:45<00:00,  3.63it/s]



Step 3000, loss_near_surface: 0.007756825536489487, loss_random: 0.14905709028244019


Avg time in seconds: 0.2735681664148967
Memory usage in Kb: 912.0009765625


In [67]:
obj_idx = 4
models_metrics[f'obj_{obj_idx}'] = run_model(obj_idx)

  0%|          | 0/3000 [00:00<?, ?it/s]



Step 0, loss_near_surface: 1.07588529586792, loss_random: -1.0




  2%|▏         | 49/3000 [00:06<03:42, 13.27it/s]



Step 50, loss_near_surface: 0.4563441574573517, loss_random: 0.582192063331604




  3%|▎         | 99/3000 [00:12<03:35, 13.44it/s]

...



Step 2900, loss_near_surface: 0.004689332097768784, loss_random: 0.05290650203824043




 98%|█████████▊| 2949/3000 [06:08<00:03, 13.33it/s]



Step 2950, loss_near_surface: 0.004601047839969397, loss_random: 0.05330970138311386




100%|██████████| 3000/3000 [06:14<00:00,  8.01it/s]



Step 3000, loss_near_surface: 0.0047919610515236855, loss_random: 0.05281751975417137


Avg time in seconds: 0.12403309591611227
Memory usage in Kb: 912.0009765625


In [68]:
obj_idx = 5
models_metrics[f'obj_{obj_idx}'] = run_model(obj_idx)

  0%|          | 0/3000 [00:00<?, ?it/s]



Step 0, loss_near_surface: 0.8217058181762695, loss_random: -1.0




  2%|▏         | 50/3000 [00:11<05:08,  9.57it/s]



Step 50, loss_near_surface: 0.2588573396205902, loss_random: 0.8039212226867676




  3%|▎         | 99/3000 [00:21<04:44, 10.21it/s]

...



Step 2900, loss_near_surface: 0.006171172484755516, loss_random: 0.14788082242012024




 98%|█████████▊| 2949/3000 [10:10<00:04, 10.89it/s]



Step 2950, loss_near_surface: 0.005839172750711441, loss_random: 0.15368574857711792




100%|██████████| 3000/3000 [10:21<00:00,  4.83it/s]



Step 3000, loss_near_surface: 0.006127447821199894, loss_random: 0.14790330827236176


Avg time in seconds: 0.20579076512654623
Memory usage in Kb: 912.0009765625


## Test models

In [85]:
def compute_occupancy_f1(points, sdf, true_sdf_val):
  preds_np = sdf(torch.cuda.FloatTensor(points.copy())).detach().cpu().numpy().flatten()
  results = np.sign(true_sdf_val) == np.sign(preds_np)
  return results.sum() / results.shape[0]


def test_sdf(sdf, obj_idx, mesh):
  points_near_surface, sdf_val_near_surface = get_near_surface_points_and_truth(mesh, num_points=5_000)
  points_random, sdf_val_random = get_random_points_and_truth(mesh, num_points=5_000)

  f1_near_surface = compute_occupancy_f1(points_near_surface, sdf, sdf_val_near_surface)
  print('Occupancy F1 near surface: ', f1_near_surface)

  f1_random = compute_occupancy_f1(points_random, sdf, sdf_val_random)
  print('Occupancy F1 random: ', f1_random)

  return f1_near_surface, f1_random

In [86]:
def test_obj_model(models_metrics, obj_idx):
  obj_siren = SirenNet(
      dim_in = 3,
      dim_hidden = 240,
      dim_out = 1,
      num_layers = 5,
      final_activation = nn.Identity(),
      w0_initial = 30
  )
  model_save_path = DATA_DIR + f'siren_model/obj_{obj_idx}.pt'
  obj_siren = obj_siren.cuda()

  mesh = trimesh.load_mesh(DATA_DIR + f'test_task_meshes/{obj_idx}.obj')
  f1_near_surface, f1_random = test_sdf(obj_siren, obj_idx, mesh)

  models_metrics[f'obj_{obj_idx}']['f1_near_surface'] = f1_near_surface
  models_metrics[f'obj_{obj_idx}']['f1_random'] = f1_random

  return models_metrics

In [87]:
models_metrics = test_obj_model(models_metrics, obj_idx=1)

Occupancy F1 near surface:  0.5472
Occupancy F1 random:  0.522


In [92]:
models_metrics = test_obj_model(models_metrics, obj_idx=2)

Occupancy F1 near surface:  0.6034
Occupancy F1 random:  0.6262


In [96]:
models_metrics = test_obj_model(models_metrics, obj_idx=3)

Occupancy F1 near surface:  0.515
Occupancy F1 random:  0.5994


In [103]:
models_metrics = test_obj_model(models_metrics, obj_idx=4)

Occupancy F1 near surface:  0.5606
Occupancy F1 random:  0.527


In [106]:
models_metrics = test_obj_model(models_metrics, obj_idx=5)

Occupancy F1 near surface:  0.5822
Occupancy F1 random:  0.6722


## Calculate average metrics

In [107]:
iter_times = []
model_memory_sizes = []
f1_near_surface_vals = []
f1_random_vals = []

for obj_idx in models_metrics.keys():
  obj_metrics = models_metrics[obj_idx]

  f1_near_surface_vals.append(obj_metrics['f1_near_surface'])
  f1_random_vals.append(obj_metrics['f1_random'])
  iter_times.append(obj_metrics['avg_time_secs'])
  model_memory_sizes.append(obj_metrics['memory_usage'])

In [108]:
def print_lst_avg(title, lst):
  print(f'Average {title} based {len(lst)} objects: ', round(sum(lst) / len(lst), 4))

print_lst_avg('iteration time (in seconds)', iter_times)
print_lst_avg('memory usage (in Kb)', model_memory_sizes)
print_lst_avg('occupancy F1 random', f1_random_vals)
print_lst_avg('occupancy F1 near surface', f1_near_surface_vals)

Average iteration time (in seconds) based 5 objects:  0.1844
Average memory usage (in Kb) based 5 objects:  912.001
Average occupancy F1 random based 5 objects:  0.5894
Average occupancy F1 near surface based 5 objects:  0.5617


In [110]:
import json

with open('/content/drive/MyDrive/UCU/4course_1term/Computer_Vision/Practice/data/results/Siren_metrics.json', 'w') as fp:
    json.dump(models_metrics, fp, indent=4)